# Chebyshev polynomials and fitting workflows

Anton Antonov  
June 2024   
December 2024

-----

## Introduction

Let us list the full set of features and corresponding packages:

- ["JavaScript::Google::Charts"](https://raku.land/zef:antononcube/JavaScript::Google::Charts)
    - Scatter plots
    - Time series data visualization
- ["Math::Polynomial::Chebyshev"](https://raku.land/zef:antononcube/Math::Polynomial::Chebyshev)
    - Polynomial basis
    - Both recursive and trigonometric methods of computation
    - The recursive method provides exact (bignum) integers for the numerators and denominators
- ["Math::Fitting"](https://raku.land/zef:antononcube/Math::Fitting)
    - Linear regression (i.e. fitting) with function bases
    - Gives functors as results
    - Multiple properties of the functors can be retrieved

- ["Data::TypeSystem"](https://raku.land/zef:antononcube/Data::TypeSystem)
    - Summary of data types

- ["Data::Summarizers"](https://raku.land/zef:antononcube/Data::Summarizers)
    - Summary of data values

### TL;DR

- Chebyshev polynomials can be exactly computed
- The "Math::Fitting" package produces functors
- The fitting is done with a function basis
- Matrix formulas are used to compute the fit (linear regression)
- Real life example is shown with weather temperature data 
    - You can just see the section before the last.

-----

## Setup

In [14]:
use Math::Matrix;
use Math::Polynomial::Chebyshev;
use Math::Fitting;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::Importers;

use JavaScript::D3;
use JavaScript::Google::Charts;

use Hash::Merge;
use LLM::Configurations;

### Google Charts

In [ ]:
#% javascript
google.charts.load('current', {'packages':['corechart']});
google.charts.load('current', {'packages':['gauge']});
google.charts.load('current', {'packages':['wordtree']});
google.charts.load('current', {'packages':['geochart']});
google.charts.load('current', {'packages':['table']});
google.charts.load('current', {'packages':['line']});
google.charts.setOnLoadCallback(function() {
    console.log('Google Charts library loaded');
});


#### Dark mode

In [ ]:
my $format = 'html';
my $titleTextStyle = { color => 'Ivory', fontSize => 16 };
my $backgroundColor = '#1F1F1F';
my $legendTextStyle = { color => 'Silver' };
my $legend = { position => "none", textStyle => {fontSize => 14, color => 'Silver'} };

my $hAxis = { title => 'x', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => False, format => 'decimal'};
my $vAxis = { title => 'y', titleTextStyle => { color => 'Silver' }, textStyle => { color => 'Gray'}, logScale => False, format => 'decimal'};

my $annotations = {textStyle => {color => 'Silver', fontSize => 10}};
my $chartArea = {left => 50, right => 50, top => 50, bottom => 50, width => '90%', height => '90%'};

#### Light mode

In [ ]:
my $format = 'html';
my $titleTextStyle = { color => 'DimGray', fontSize => 16 };
my $backgroundColor = 'White';
my $legendTextStyle = { color => 'DarkGray' };
my $legend = { position => "none", textStyle => {fontSize => 14, color => 'DarkGray'} };

my $hAxis = { title => 'x', titleTextStyle => { color => 'DimGray' }, textStyle => { color => 'DarkGray'}, logScale => False, format => 'decimal'};
my $vAxis = { title => 'y', titleTextStyle => { color => 'DimGray' }, textStyle => { color => 'DarkGray'}, logScale => False, format => 'decimal'};

my $annotations = {textStyle => {color => 'DarkGray', fontSize => 10}};
my $chartArea = {left => 50, right => 50, top => 50, bottom => 50, width => '90%', height => '90%'};

-------

## Computation granularity

In [ ]:
chebyshev-t(3, 0.3)

In [ ]:
my $k = 12;

# Whatever goes to 'recursive'
my $method = 'recursive'; # 'trig'

my @x = (-1.0, -0.99 ... 1.0);
say '@x.elems : ', @x.elems;

my @data  = @x.map({ [$_, chebyshev-t($k, $_, :$method)]});
my @data1 = chebyshev-t($k, @x);

say deduce-type(@data);
say deduce-type(@data1);

In [ ]:
sink records-summary(@data.map(*.tail) <<->> @data1)

-----

## Precision

We can compute the exact Chebyshev polynomial values at given points using `FatRat` numbers:

In [ ]:
my $v = chebyshev-t(100, <1/4>.FatRat, method => 'recursive')

Here are the numerator and denominator:

In [ ]:
say $v.numerator;
say $v.denominator;

-----

## Plots

This section shows how the plot the Chebyshev polynomials using [Google Charts](https://developers.google.com/chart) via ["JavaScript::Google::Charts"](https://raku.land/zef:antononcube/JavaScript::Google::Charts).

### Single polynomial

Single polynomial plot using a [Line chart](https://developers.google.com/chart/interactive/docs/gallery/linechart):

In [ ]:
#%html
my $n = 6;
my @data = chebyshev-t(6, (-1, -0.98 ... 1).List);
js-google-charts('LineChart', @data, 
    title => "Chebyshev-T($n) polynomial", 
    :$titleTextStyle, :$backgroundColor, :$chartArea, :$hAxis, :$vAxis,
    width => 800, 
    div-id => 'poly1', :$format,
    :png-button)

### Basis

Doing fitting we are interested in using bases of functions. Here for first eight Chebyshev-T polynomials make plot data:

In [ ]:
my $n = 8;
my @data = (-1, -0.98 ... 1).map(-> $x { [x => $x, |(0..$n).map({ $_.Str => chebyshev-t($_, $x, :$method) }) ].Hash });

deduce-type(@data):tally;

Here is the plot with all eight functions:

In [ ]:
#%html
js-google-charts('LineChart', @data,
    column-names => ['x', |(0..$n)».Str],
    title => "Chebyshev T polynomials, 0 .. $n",
    :$titleTextStyle,
    width => 800, 
    height => 400,
    :$backgroundColor, :$hAxis, :$vAxis,
    legend => merge-hash($legend, %(position => 'right')),
    chartArea => merge-hash($chartArea, %(right => 100)),
    format => 'html', 
    div-id => "cheb$n",
    :$format,
    :png-button)

-----

## Text plot

*Text plots always work!*

In order to plot with ["Text::Plot"](https://raku.land/zef:antononcube/Text::Plot) 
the data has to be converted into [long form](https://en.wikipedia.org/wiki/Wide_and_narrow_data) first:

In [ ]:
my @dataLong = to-long-format(@data, <x>).sort(*<Variable x>);
deduce-type(@dataLong):tally

Here is a sample:

In [ ]:
#% html
@dataLong.pick(10)
==> {.sort(*<Variable x>)}()
==> to-html(field-names => <Variable x Value>)

Here is the text plot:

In [ ]:
my @chebInds = 1, 2, 3, 4;
my @dataLong3 = @dataLong.grep({ $_<Variable>.Int ∈ @chebInds }).classify(*<Variable>).map({ $_.key => $_.value.map(*<x Value>).Array }).sort(*.key)».value;
say @chebInds Z=> <* □ ▽ ❍>; 
text-list-plot(@dataLong3, width => 100, height => 25, title => "Chebyshev T polynomials, 0 .. $n")

-----

## Fitting

Here we generate "measurements data" with noise:

In [ ]:
my @temptimelist = 0.1, 0.2 ... 20;
my @tempvaluelist = @temptimelist.map({ sin($_) / $_ }) Z+ (1..200).map({ (3.rand - 1.5) * 0.02 });
my @data1 = @temptimelist Z @tempvaluelist;
@data1 = @data1.deepmap({ .Num });

deduce-type(@data1)

Rescaling of the x-coordinates:

In [ ]:
my @data2 = @data1.map({ my @a = $_.clone; @a[0] = @a[0] / max(@temptimelist); @a });

deduce-type(@data2)

Here is a summary:

In [ ]:
sink records-summary(@data2)

Here is a plot of that data:

In [ ]:
#% html
js-google-charts("Scatter", @data2, 
    title => 'Measurements data with noise',
    :$backgroundColor, :$hAxis, :$vAxis,
    :$titleTextStyle, :$chartArea,
    width => 800, 
    div-id => 'data', :$format,
    :png-button)

Make a function that rescales from $[0, 1]$ to $[-1, 1]$:

In [ ]:
my &rescale = { ($_ - 0.5) * 2 };

Here is a list of basis functions:

In [ ]:
my @basis = (^16).map({ chebyshev-t($_) o &rescale });
@basis.elems

**Remark:** Function composition operator `o` is used above. Before computing the Chebyshev polynomial value the argument is rescaled.

Here we compute a linear model fit with those functions:

In [ ]:
my &lm = linear-model-fit(@data2, :@basis)

Here are the best fit parameters:

In [ ]:
&lm('BestFitParameters')

Here is a plot of those parameters:

In [ ]:
#% html
js-google-charts("Bar", &lm('BestFitParameters'), 
    :!horizontal,
    title => 'Best fit parameters',
    :$backgroundColor, 
    hAxis => merge-hash($hAxis, {title => 'Basis function index'}), 
    vAxis => merge-hash($hAxis, {title => 'Coefficient'}), 
    :$titleTextStyle, :$chartArea,
    width => 800, 
    div-id => 'bestFitParams', :$format,
    :png-button)

We can see from the plot that using more the 12 basis functions for that data is not improving the fit, since the coefficients after the 12th index are very small.

Now, let us plot the data and the fit. First we prepare the plot data:

In [ ]:
my @fit = @data2.map(*.head)».&lm;
my @plotData = transpose([@data2.map(*.head).Array, @data2.map(*.tail).Array, @fit]);
@plotData = @plotData.map({ <x data fit>.Array Z=> $_.Array })».Hash;

deduce-type(@plotData)

Here is the plot:

In [ ]:
#% html
js-google-charts('ComboChart', 
    @plotData, 
    title => 'Data and fit',
    column-names => <x data fit>,
    :$backgroundColor, :$titleTextStyle :$hAxis, :$vAxis,
    seriesType => 'scatter',
    series => {
        0 => {type => 'scatter', pointSize => 2, opacity => 0.1, color => 'Gray'},
        1 => {type => 'line'}
    },
    legend => merge-hash($legend, %(position => 'bottom')),
    :$chartArea,
    width => 800, 
    div-id => 'fit1', :$format,
    :png-button)

Compute the residuals of the last fit:

In [ ]:
sink records-summary( (@fit <<->> @data2.map(*.tail))».abs )

----

## Condition number

The formula with which the [Ordinary Least Squares (OLS)](https://en.wikipedia.org/wiki/Ordinary_least_squares) fit is computed is:

$$
\beta = (X^T \cdot X)^{-1} \cdot X^T \cdot y
$$

Let us look into the condition number of the "normal matrix" (or "Gram matrix") $X^T \cdot X$ . First, we get the design matrix:

In [ ]:
my @a = &lm.design-matrix();
my $X = Math::Matrix.new(@a);
$X.size

Here is the Gram matrix:

In [ ]:
my $g = $X.transposed dot $X;
$g.size

And here is the [condition number](https://en.wikipedia.org/wiki/Condition_number) of that matrix:

In [ ]:
$g.condition

We conclude that we are fine to use that design matrix.

**Remark:** For a system of linear equations in matrix form $A x = b$, the condition number of $A$, $\kappa (A)$, is defined to be the maximum ratio of the relative error in $x$ to the relative error in $b$.

**Remark:** Typically, if the condition number is $\kappa (A)=10^{d}$, we can expect to lose as many as $d$ digits of accuracy 
in addition to any loss caused by the numerical method (due to precision issues in arithmetic calculations.)

**Remark:** A very "Raku-way" to define ill-conditioned matrix as "almost is not of full rank," or "if its inverse does not exist."
 

-----

## Temperature data

Let us redo the whole workflow with a real life data -- weather temperature data for 4 consecutive years of Greenville, South Carolina, USA. 
(Where the [Perl and Raku Conference 2025](https://www.perl.com/article/get-ready-for-the-2025-perl-and-raku-conference/) is going to be held.)

Here we ingest the time series data:

In [ ]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/dsTemperature-Greenville-SC-USA.csv';
my @dsTemperature = data-import($url, headers => 'auto');
@dsTemperature = @dsTemperature.deepmap({ $_ ~~ / ^ \d+ '-' / ?? DateTime.new($_) !! $_.Num });
deduce-type(@dsTemperature)

Show data summary:

In [ ]:
sink records-summary(@dsTemperature, field-names => <Date AbsoluteTime Temperature>)

Here is a plot:

In [ ]:
#% html
js-google-charts("Scatter", @dsTemperature.map(*<Date Temperature>), 
    title => 'Temperature of Greenville, SC, USA',
    :$backgroundColor,
    hAxis => merge-hash($hAxis, {title => 'Time', format => 'M/yy'}), 
    vAxis => merge-hash($hAxis, {title => 'Temperature, ℃'}), 
    :$titleTextStyle, :$chartArea,
    width => 1200, 
    height => 400, 
    div-id => 'tempData', :$format,
    :png-button)

Here is a fit -- note the rescaling:

In [ ]:
my ($min, $max) = @dsTemperature.map(*<AbsoluteTime>).Array.&{ (.min, .max) }();

In [ ]:
my &rescale-time = { -($max + $min) / ($max - $min) + (2 * $_) / ($max - $min)};
my @basis = (^16).map({ chebyshev-t($_) o &rescale-time });
@basis.elems

In [ ]:
my &lm-temp = linear-model-fit(@dsTemperature.map(*<AbsoluteTime Temperature>), :@basis)

Her is a plot of the time series and the fit:

In [ ]:
my @fit = @dsTemperature.map(*<AbsoluteTime>)».&lm-temp;
my @plotData = transpose([@dsTemperature.map({ $_<AbsoluteTime> }).Array, @dsTemperature.map(*<Temperature>).Array, @fit]);
@plotData = @plotData.map({ <x data fit>.Array Z=> $_.Array })».Hash;

deduce-type(@plotData)

In [ ]:
#% html

my @ticks = @dsTemperature.map({ %( v => $_<AbsoluteTime>, f => $_<Date>.Str.substr(^7)) })».Hash[0, 120 ... *];

js-google-charts('ComboChart', 
    @plotData,
    title => 'Temperature data and Least Squares fit',
    column-names => <x data fit>,
    :$backgroundColor, :$titleTextStyle,
    hAxis => merge-hash($hAxis, {title => 'Time', :@ticks, textPosition => 'in'}), 
    vAxis => merge-hash($hAxis, {title => 'Temperature, ℃'}), 
    seriesType => 'scatter',
    series => {
        0 => {type => 'scatter', pointSize => 3, opacity => 0.1, color => 'Gray'},
        1 => {type => 'line', lineWidth => 4}
    },
    legend => merge-hash($legend, %(position => 'bottom')),
    :$chartArea,
    width => 1200, 
    height => 400, 
    div-id => 'tempDataFit', :$format,
    :png-button)

-----

## Future plans

At this point it should be clear that Raku is fully equipped to do regression analysis for both didactical and "real-life" purposes.

I plan to implement in Raku next year the necessary computational frameworks to do [Quantile Regression](https://en.wikipedia.org/wiki/Quantile_regression).

The workflow code in this post can be generated using LLMs -- I plan to write about that soon.